# Set the Hue

In [15]:
from phue import Bridge

# https://www.meethue.com/api/nupnp
b = Bridge('192.168.1.7')

In [20]:
# Switch light 3 on
b.set_light(1,'on', False)

[[{'success': {'/lights/1/state/on': False}}]]

In [22]:
b.set_light(2,'on', False)

[[{'success': {'/lights/2/state/on': False}}]]

In [19]:
# Switch light 3 on
b.set_light(1,'on', True)
b.set_light(2,'on', True)
b.set_light(3,'on', True)

[[{'success': {'/lights/3/state/on': True}}]]

In [119]:
# Switch light 3 on
b.set_light(3,'on', True)

[[{'success': {'/lights/3/state/on': True}}]]

In [3]:
# Prints if light 1 is on or not
b.get_light(3, 'on')

True

In [7]:
# Set brightness of lamp 3 to 100%
b.set_light(3, 'bri', 255)

[[{'success': {'/lights/3/state/bri': 254}}]]

In [5]:
# Switch light 3 off
b.set_light(3,'on', False)

[[{'success': {'/lights/3/state/on': False}}]]

# Voice first application

In [28]:
import sounddevice as sd
import soundfile as sf
from bs4 import BeautifulSoup
import speech_recognition as sr_audio 
import os, pyttsx3, pygame, time
import speech_recognition as sr_audio 

r = sr_audio.Recognizer()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [35]:
def sync_record(filename, duration, fs, channels):
    print('recording')
    myrecording = sd.rec(int(duration * fs), samplerate=fs, channels=channels) 
    sd.wait()
    sf.write(filename, myrecording, fs)
    print('done recording')
    
def sync_playback(filename):
    # takes in a file and plays it back 
    pygame.mixer.init() 
    pygame.mixer.music.load(filename) 
    pygame.mixer.music.play()
    
def speak_text(text): 
    engine=pyttsx3.init() 
    engine.setProperty('voice', 'com.apple.speech.synthesis.voice.Alex')
    engine.say(text) 
    engine.runAndWait()
    
def transcribe_audio_sphinx(filename):
# transcribe the audio (note this is only done if a voice sample) r=sr_audio.Recognizer()
    with sr_audio.AudioFile(filename) as source:
        audio = r.record(source)
        
    text=r.recognize_sphinx(audio) 
    print('transcript: '+text)
    return text
    
#speak_text('What do you want to search?')


In [52]:
sync_record('response.wav',2,16000,1) 
transcript=transcribe_audio_sphinx('response.wav')

if transcript.lower().find('hello') >= 0:
    b.set_light(3,'on', True)
    
if transcript.lower().find('bye') >= 0:
    b.set_light(3,'on', False)

recording
done recording
transcript: the other and all


In [40]:
if transcript.lower().find('hello') >= 0:
    b.set_light(3,'on', True)

In [53]:
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums
import io

def sample_long_running_recognize(local_file_path):
    """
    Transcribe a long audio file using asynchronous speech recognition
    Args:
      local_file_path Path to local audio file, e.g. /path/audio.wav
    """

    client = speech_v1.SpeechClient()

    # local_file_path = 'resources/brooklyn_bridge.raw'

    # The language of the supplied audio
    language_code = "fr-FR"

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 16000

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = enums.RecognitionConfig.AudioEncoding.LINEAR16
    config = {
        "language_code": language_code,
        "sample_rate_hertz": sample_rate_hertz,
        "encoding": encoding,
    }
    with io.open(local_file_path, "rb") as f:
        content = f.read()
    audio = {"content": content}

    operation = client.long_running_recognize(config, audio)

    print(u"Waiting for operation to complete...")
    response = operation.result()

    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))



In [54]:
sample_long_running_recognize('response.wav')

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

# Face detection

In [7]:
import cv2
import numpy as np
import time
from phue import Bridge
import dlib
from imutils import face_utils
face_detect = dlib.get_frontal_face_detector()

# https://www.meethue.com/api/nupnp
b = Bridge('192.168.1.90')

In [8]:
video_capture = cv2.VideoCapture(0)
flag = 0

while True:

    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = face_detect(gray, 1)

    if len(rects) == 0:
        b.set_light(3,'on', False)
    else:
        b.set_light(3,'on', True)
        for (i, rect) in enumerate(rects):

            (x, y, w, h) = face_utils.rect_to_bb(rect)

            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            cv2.imshow('Video', frame)
    
    time.sleep(1)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

# Tap detection

In [112]:
myrecording = sd.rec(int(1 * 16000), samplerate=16000, channels=1) 
sd.wait()
sf.write('tap.wav', myrecording, fs)

In [113]:
y1, sr1 = librosa.load('tap.wav')
y2, sr2 = librosa.load('tap2.wav')

mfcc1 = librosa.feature.mfcc(y1, sr1)
mfcc2 = librosa.feature.mfcc(y2, sr2)

In [114]:
dist, _, _, _ = dtw(mfcc1.T, mfcc2.T, dist=lambda x, y: norm(x - y, ord=1))

In [115]:
dist

46888.411697387695

In [92]:
import librosa
from numpy.linalg import norm

record_time = 1
fs = 16000
y1, sr1 = librosa.load('tap.wav')
mfcc1 = librosa.feature.mfcc(y1, sr1)
    
while True:
    
    myrecording = sd.rec(int(record_time * fs), samplerate=fs, channels=1) 
    sd.wait()
    sf.write('tap2.wav', myrecording, fs)

    y2, sr2 = librosa.load('tap2.wav')
    mfcc2 = librosa.feature.mfcc(np.array(myrecording).reshape(1,-1)[0], fs)

    dist, _, _, _ = dtw(mfcc1.T, mfcc2.T, dist=lambda x, y: norm(x - y, ord=1))
    print(dist)
    if dist < 10000:
        print(True)

12830.320281982422
12921.189086914062
13123.377685546875
10173.62612915039
17451.760864257812
14924.678405761719
13308.22671508789
18953.690979003906
11359.824798583984
15054.652221679688
18459.833587646484
13191.686706542969
15986.970306396484
17152.84197998047
16471.7430267334
16089.865112304688
17632.661224365234


KeyboardInterrupt: 

In [72]:
dist, cost, acc_cost, path = dtw(mfcc1.T, mfcc2.T, dist=lambda x, y: norm(x - y, ord=1))

In [73]:
dist

28637.941162109375

In [22]:
manhattan_distance = lambda data1, data2: np.abs(data1 - data2)

d, cost_matrix, acc_cost_matrix, path = dtw(data1, data2, dist=manhattan_distance)

KeyboardInterrupt: 

In [16]:
import numpy as np

# We define two sequences x, y as numpy array
# where y is actually a sub-sequence from x
x = np.array([2, 0, 1, 1, 2, 4, 2, 1, 2, 0]).reshape(-1, 1)
y = np.array([1, 1, 2, 4, 2, 1, 2, 0]).reshape(-1, 1)

from dtw import dtw

manhattan_distance = lambda x, y: np.abs(x - y)

d, cost_matrix, acc_cost_matrix, path = dtw(x, y, dist=manhattan_distance)

print(d)

2.0


In [ ]:
myrecording = sd.rec(int(duration * fs), samplerate=fs, channels=channels) 
sd.wait()
sf.write(filename, myrecording, fs)
print('done recording')